In [90]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain import HuggingFaceHub

load_dotenv(find_dotenv())

True

In [91]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}
"""

llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_length":256, "max_new_tokens":100})

prompt_template = PromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I ordered Pizza Salami and it was awesome!")

'{\n  "sentiment": "positive",\n  "subject": "Pizza Salami"\n}'

## Sequentials Chains
Sometimes you want to pass the output from one model to a another model. This can be done with different SequentialsChains

In [92]:
response_template = """
You are a helpful bot that creates 'thank you' response messages based on the sentiment and the subject of the review.You will get a sentiment and subject as inputs to evaluate in json format.

If the sentiment is negative, apologize to the customer and offer him a real world assistant to talk to. 

just return the message you created as plain text.

DO NOT INCLUDE ANYTHING ELSE IN YOUR RESPONSE.

input: {input}
"""
review_template = PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=llm, prompt=review_template)

In [93]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="I ordered Pizza Salami and was great!")



> Entering new SimpleSequentialChain chain...
{
  "sentiment": "positive",
  "subject": "Pizza Salami"
}

output:

Thank you for taking the time to review our product. We appreciate your feedback and are glad to hear that you enjoyed our Pizza Salami. We strive to provide the best quality products and customer service, and we're glad to hear that you had a positive experience with us. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help.

> Finished chain.


"\noutput:\n\nThank you for taking the time to review our product. We appreciate your feedback and are glad to hear that you enjoyed our Pizza Salami. We strive to provide the best quality products and customer service, and we're glad to hear that you had a positive experience with us. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help."

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [94]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You are a helpful bot that creates reviews based on the dish name and the experience. You ordered {dish_name} and your experience was {experience}. Write a review with words: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="You are a helpful bot that creates follow-up comments for reviews.Your comments have to be only one sentence long. Given the review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {review}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")


overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary],
    verbose=True,
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary",],
)

overall_chain({"dish_name": "Pizza Salami", "experience": "It was awful!"})



> Entering new SequentialChain chain...

> Finished chain.


{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': '"The pizza was disappointing, with a soggy crust and bland toppings. The salami was tough and dry, and the overall taste was unpalatable. I would not recommend this pizza to anyone."',
 'comment': '"I\'m sorry to hear that you didn\'t enjoy your pizza. We strive to provide the best quality products, and we\'ll make sure to address any concerns you have. We hope you\'ll give us another chance to make it right."',
 'summary': '\nThe review describes a pizza with a disappointing taste due to a soggy crust, tough and dry toppings, and overall unpalatable taste.'}

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

we start by defining 3 templates, a positive, negative and neutral one

In [95]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [96]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"max_length":256, "max_new_tokens":100})


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. your output should be the highlights of the text.\nHere is the text:\n{input}', template_format='f-string', validate_template=True), llm=HuggingFaceHub(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text2text-generation/google/flan-t5-xxl', task='text2text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='google/flan-t5-xxl', task=None, model_kwargs={'max_length': 256, 'max_new_tokens': 100}, huggingfacehub_api_token=None), output_key='text', output_parser=StrOutputParser(), return_final_only

here we have to do some work to get the chain output right.
First:
We need to create a custom OutPutParser for the router chain, this is due to a small bug in the RouterOutPutParser abstraction in langchain, presumably it has been fixed in newer versions. But in addition to that we have an issue regarding the output format, this is mostly because the free and open source model we are using is not as good as the commercial ones. So we need to do some work to get the output right.
The only thing we changed in the parse method is how we load the Json, from te text input that has been previously concatenated to curly braces, just as JSON format needs it.

In [97]:
import json
import langchain
from typing import Any, Dict, List, Optional, Type, cast

class CustomRouterOutputParser(langchain.schema.BaseOutputParser[Dict[str, str]]):
    """Parser for output of router chain in the multi-prompt chain."""

    default_destination: str = "DEFAULT"
    next_inputs_type: Type = str
    next_inputs_inner_key: str = "input"

    def parse(self, text: str) -> Dict[str, Any]:
        try:
            #! bc our model is having a hard time getting json format right, we will just add the curly braces here
            if text[-1] != "}": text = text + "}"
            if text[0] != "{": text = "{" + text
            print("parsed text",text)
            parsed = json.loads(text)
            if not isinstance(parsed["destination"], str):
                raise ValueError("Expected 'destination' to be a string.")
            if not isinstance(parsed["next_inputs"], self.next_inputs_type):
                raise ValueError(
                    f"Expected 'next_inputs' to be {self.next_inputs_type}."
                )
            parsed["next_inputs"] = {self.next_inputs_inner_key: parsed["next_inputs"]}
            if (
                parsed["destination"].strip().lower()
                == self.default_destination.lower()
            ):
                parsed["destination"] = None
            else:
                parsed["destination"] = parsed["destination"].strip()
            return parsed
        except Exception as e:
            raise Exception(
                f"Parsing text\n{text}\n raised following error:\n{e}"
            )

Now we need to dive deep into the Router Template and do some prompt engineering to it.
The router Template contains a bunch of instructions and format directives that we need to enhance in order to make the model behave as we want to.

In [99]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations) + "\n"
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str).replace("<< OUTPUT >>","<< OUTPUT (must only be a json object) >>").replace("""<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```""","""<< FORMATTING >>
Return a JSON object formatted to look like:
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
{{
"destination": "physics",
"next_inputs": "What is black body radiation?"
}}
""")
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=CustomRouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.run(input="I ordered Pizza Salami for 9.99$ and I hated it, the crust is awful!")



> Entering new MultiPromptChain chain...


/Users/atorres/.local/share/virtualenvs/langchain-kv9OLnAB-python/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


parsed text {"destination": "negative", "next_inputs": "I ordered Pizza Salami for 9.99$ and I hated it, the crust is awful!"}
negative: {'input': 'I ordered Pizza Salami for 9.99$ and I hated it, the crust is awful!'}
> Finished chain.


'The crust is awful'